# 虚谷号WebGPIO应用（客户端Python版）

虚谷号和手机（App inventor）如何互动控制？

虚谷号和掌控板如何互动控制？

为了让虚谷号和其他开源硬件、编程语言快速互动，虚谷号的WebGPIO应运而生。简单的说，只要在虚谷号上运行一个python文件，就可以用WebAPI的形式来与虚谷号互动，可以获取虚谷号板载Arduino的所有引脚的电平，也可以控制所有引脚。

## 1.接口介绍

要在虚谷号上运行“webgpio.py”。也可以将“webgpio.py”文件更名为“main.py”，复制到vvBoard的Python目录，只要一开机，虚谷号就会执行。

下载地址：https://github.com/vvlink/vvBoard-docs/tree/master/webgpio

WebAPI地址：

http://[虚谷号ip]:1024/

注：下面假设虚谷号的IP地址为：192.168.1.101

### 1.1 获取引脚状态

method方式:GET

参数示例： { pin:"D2" }

url范例：http://192.168.1.101:1024/?pin=D2

信息返回：

当pin为D0--D13时，读取数字引脚的数字值，0为低电平，1为高电平。

{ "pin":"D1", "error_code":0, "msg":1 }

当pin为A0--A5时，读取模拟引脚的模拟值，0-255之间。

{ "pin":"A0", "error_code":0, "msg":255 }

### 1.2. 控制引脚电平

method方式: POST

参数示例：

{ pin:"D1" value:255 type:"digital" }

注：Digital、Analog、Servo等词语不分大小写，也可以用“1、2、3”等数字来代替。

 - 当type为digital时，设置引脚的电平值为value的值，0表示LOW，非0表示HIGH；
 - 当type为analog时，设置引脚的PWM值为value的值，即0-255之间；
 - 当type为servo时，设置引脚上舵机的转动角度为value的值，即0-180之间。


返回参数：

{ "pin":"D2", "error_code":0, "msg":"success,set [pin] to [value] with [types] mode" }

当pin不在D0--D13，A0--A5之间时：

{ "pin":"D2", "error_code":1 "msg":"error,invalid Pin" }

当value不能转换整数时：

{ "pin":"D2", "error_code":1, "msg":"error,Value is wrong" }

当type不正确时：

{ "pin":"D2", "error_code":1, "msg":"error,Type is wrong" }

## 2. 客户端代码范例（Python）

虽然通过任何一个能够发送Http请求的工具，包括浏览器、Word、掌控板、手机等，都可以和虚谷号互动。接下来选择Python语言写一个Demo代码。Python借助Requests库来发送Http请求，是非常方便的。参数传递方面，同时支持params和data两种模式。

### 2.1.调用POST方法，对虚谷号的引脚进行控制

在该案例中可以修改的参数有:

    - url:设置成虚谷号的IP地址
    - pin:对应的引脚 A0-A5，D0-D13 
    - value:对应的数值
    - type:控制的类型可以是1，2，3，分别代表“digital”、“analog”、“servo”
    
当设置D13号引脚的电平为1，该引脚对应的LED就会亮起。

In [1]:
import requests

vvboardip='192.168.3.42'
pin='D13'
value=1
t=1
payload = {"pin":pin,'value':value,'type':t}
re = requests.post(url='http://'+ vvboardip +':1024/',params=payload) 
if (re.status_code==200):
    r=re.json()
    print('成功发送控制命令:'+ r["msg"]) 
    print('返回的信息为：') 
    print(re.text) 

成功发送控制命令:success,set D13 to 1 with 1 mode
返回的信息为：
{
    "error_code": 0,
    "msg": "success,set D13 to 1 with 1 mode",
    "pin": "D13"
}


### 2.2. 调用GET方法，读取A0号引脚的电平。

在该案例中可以修改的参数有:

    - url:设置成虚谷号的IP地址
    - pin:对应的引脚 A0-A5，D0-D13 
    
注意：该方法需要外接传感器，否则数字口默认返回为低电平，模拟口返回随机数。

In [2]:
import requests

vvboardip='192.168.3.42'
pin='A0'
payload = {"pin":pin}
re = requests.get(url='http://'+ vvboardip +':1024/',params=payload) 
if (re.status_code==200):
    r=re.json()
    print('成功获取引脚'+ r["pin"] + '的状态:'+ r["msg"]) 
    print('返回的原始信息为：') 
    print(re.text) 

成功获取引脚A0的状态:393
返回的原始信息为：
{
    "error_code": 0,
    "msg": "393",
    "pin": "A0"
}


## 3. 其他说明

1.手机上快速控制，如何实现？

访问：http://192.168.3.42:1024/help/

可以直接在网页上测试接口。

2.App invntor如何借助这一接口与虚谷号互动？

请参考github，提供了范例。

https://github.com/vvlink/vvBoard-docs/tree/master/webgpio，

3.掌控板如何利用这一接口与虚谷号互动？

掌控板中提供了urequests库，在mPython软件中可以编写发送Http请求的应用。

另外，掌控板中提供了WebtinyIO，使用方式和虚谷号的WebGPIO基本一致。